In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re
#stopwords
import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#spit data
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/PPW/tugas

/content/drive/MyDrive/PPW/tugas


# **Implementasi Klasifikasi kategori berita**

## **Deskripsi Dataset**

Dataset ini berisi berita yang dikelompokkan ke dalam dua kategori, yaitu 'Politik' dan 'Gaya Hidup'.

Jumlah Data : 100 baris data dengan 5 kolom

Dataset ini TIDAK MEMILIKI Missing Values yang berarti bahwa setiap bagian data pada dataset lengkap dengan informasi.

## **Tujuan**

Tujuan dari dataset ini adalah untuk mengklasifikasikan berita ke dalam dua kategori, yaitu "Politik" dan "Gaya Hidup". Dengan menggunakan teknik pemrosesan teks dan model klasifikasi, seperti Logistic Regression, dataset ini bertujuan untuk membangun sistem yang dapat secara otomatis mengidentifikasi kategori berita berdasarkan konten yang diberikan.

## **Penjelasan Fitur**

Jumlah Fitur : 4

* **Judul Berita** : Judul dari setiap berita
* **Isi Berita** : Konten utama dari berita. Kolom ini memuat informasi lengkap tentang peristiwa, fakta, atau opini yang dibahas dalam berita. Fitur ini diproses lebih lanjut untuk analisis teks.
* **Tanggal Berita** : Fitur ini menunjukkan kapan berita tersebut diterbitkan berupa tanggal dan waktu
* **Kategori Berita** : Fitur ini merupakan label atau kategori dari setiap berita. Kategori ini menunjukkan apakah berita tersebut berkaitan dengan Politik atau Gaya Hidup. Fitur ini penting karena merupakan target yang akan diprediksi dalam model klasifikasi. Kolom kategori ini nantinya akan diubah menjadi nilai numerik (misalnya, 0 untuk Politik dan 1 untuk Gaya Hidup).

## **Sumber Dataset**

[https://timesindonesia.co.id/#google_vignette](https://timesindonesia.co.id/#google_vignette)

Dataset ini berisi kumpulan berita yang diambil dari situs TIMES Indonesia, sebuah portal berita yang memuat berbagai artikel terkait berbagai topik, termasuk politik dan gaya hidup. Data ini dikumpulkan melalui proses web crawling menggunakan Python, yang merupakan teknik pengumpulan data secara otomatis dari situs web.

## **Dataset**

In [7]:
df = pd.read_csv("Tugas-Crawling-Data-Berita-2-kategori.csv")

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik
...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup


## Preprocessing

### **Label Encoder**

In [8]:
df['Kategori'] = df['Kategori Berita'].map({'Politik': 0, 'Gaya Hidup': 1})

### **Cleaning**

In [10]:
def remove_url(data_berita):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', data_berita)

def remove_html(data_berita):
  html = re.compile(r'<.#?>')
  return html.sub(r'', data_berita)

def remove_emoji(data_berita):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
  data_berita = re.sub(r'\d+', '', data_berita)
  return data_berita

def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)
  return data_berita

df['cleansing'] = df['Isi Berita'].apply(lambda x: remove_url(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_html(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_emoji(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_symbols(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_numbers(x))

df['cleansing']

,cleansing
0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...
1,TIMESINDONESIA MALANG Bakal cawabup Malang La...
2,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...
3,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...
4,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...
...,...
95,TIMESINDONESIA MALANG Azquira Scarf menggelar...
96,TIMESINDONESIA MALANG Pada Minggu komunitas ...
97,TIMESINDONESIA SURABAYA Memilih wewangian pen...
98,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...


### **Case Folding**

In [11]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df['case_folding']

,case_folding
0,timesindonesia jakarta di bawah kepemimpinan ...
1,timesindonesia malang bakal cawabup malang la...
2,timesindonesia bantul rois syuriah pcnu bantu...
3,timesindonesia mojokerto ribuan kader pkb kab...
4,timesindonesia jakarta partai kebangkitan ban...
...,...
95,timesindonesia malang azquira scarf menggelar...
96,timesindonesia malang pada minggu komunitas ...
97,timesindonesia surabaya memilih wewangian pen...
98,timesindonesia magelang anda ingin mencicipi ...


### **Tokenize**

In [12]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

### **Stop Word Removal**

In [14]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df['stopword_removal']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,stopword_removal
0,timesindonesia jakarta kepemimpinan presiden r...
1,timesindonesia malang cawabup malang lathifah ...
2,timesindonesia bantul rois syuriah pcnu bantul...
3,timesindonesia mojokerto ribuan kader pkb kabu...
4,timesindonesia jakarta partai kebangkitan bang...
...,...
95,timesindonesia malang azquira scarf menggelar ...
96,timesindonesia malang minggu komunitas oom ir ...
97,timesindonesia surabaya memilih wewangian peng...
98,timesindonesia magelang mencicipi kenikmatan s...


### **Stemming**

In [15]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df['stemming'] = df['stopword_removal'].apply(lambda x: ' '.join(stemming(x.split())))

df['stemming']

,stemming
0,timesindonesia jakarta pimpin presiden ri prab...
1,timesindonesia malang cawabup malang lathifah ...
2,timesindonesia bantul rois syuriah pcnu bantul...
3,timesindonesia mojokerto ribu kader pkb kabupa...
4,timesindonesia jakarta partai bangkit bangsa p...
...,...
95,timesindonesia malang azquira scarf gelar acar...
96,timesindonesia malang minggu komunitas oom ir ...
97,timesindonesia surabaya pilih wewangian harum ...
98,timesindonesia magelang cicip nikmat deret kul...


### **Split Data**

In [16]:
x = df['stemming'].values
y = df['Kategori'].values

In [17]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=2)

## **TF-IDF**

In [19]:
# Inisialisasi TfidfVectorizer
vect = TfidfVectorizer()

# Fitting dan transformasi data pelatihan (mengajarkan model tentang kata-kata di Xtrain dan menghitung TF-IDF)
X = vect.fit_transform(Xtrain)

# Mengonversi Xtrain menjadi representasi TF-IDF
X_array = vect.transform(Xtrain)

# Mengubah matriks menjadi array
X_array.toarray()

# Menyimpan dalam pkl
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vect, f)

## **Logistic Regression**

In [28]:
# Membuat model Logistic Regression
model = LogisticRegression()

# Melatih model dengan data training
model.fit(X, Ytrain)

LogisticRegression()

In [29]:
# Meyimpan model
with open('logistic_model.pkl', 'wb') as f:
    pickle.dump(model, f)

## **Implementasi menggunakan data baru (Input User)**

In [32]:
# Load TF-IDF dan model logistic regression
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('logistic_model', 'rb') as f:
    logistic_model = pickle.load(f)

In [33]:
# memPreprosesing input pengguna
def preprocess_input(text):
    text = remove_url(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_symbols(text)
    text = remove_numbers(text)
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

In [34]:
# Untuk memprediksi kategori dari input pengguna
def predict_category(user_input):
    # Preprocessing input
    preprocessed_input = preprocess_input(user_input)

    # Transformasi input pengguna menggunakan TF-IDF
    input_tfidf = tfidf_vectorizer.transform([preprocessed_input])

    # Memprediksi Kategori menggunakan MODEL LOGISTIC REGRESSION
    prediction = logistic_model.predict(input_tfidf)

    # Memetakan presiksi kategori
    category_map = {0: 'Politik', 1: 'Gaya Hidup'}
    return category_map[prediction[0]]

In [35]:
# input pengguna
user_input = input("Masukkan isi berita: ")
predicted_category = predict_category(user_input)
print(f"Kategori berita: {predicted_category}")

Masukkan isi berita: Menjadi tugas dan tanggung jawab kita bersama, seluruh anggota DPR RI, untuk dapat menjalankan amanat rakyat dengan sebaik-baiknya," ujar Puan di hadapan para anggota dewan.  Puan menjelaskan bahwa setiap kali pemilu selesai dan rakyat telah memilih wakilnya, selalu ada harapan bahwa kekuasaan yang diberikan kepada DPR RI bisa membawa perubahan positif bagi kesejahteraan rakyat.  Namun, menurut Puan, Indonesia dihadapkan pada berbagai tantangan yang kompleks dalam upaya membangun negeri ini. Beberapa masalah struktural yang menjadi sorotan adalah kualitas sumber daya manusia (SDM), kedaulatan pangan, energi, pengembangan industri nasional, serta pemberdayaan usaha mikro, kecil, dan menengah (UMKM).  "Kita juga menghadapi ketidakpastian ekonomi global dan geopolitik, serta perekonomian nasional yang masih dalam proses pemulihan," ujarnya.  Lebih lanjut, Puan menekankan bahwa DPR RI harus menggunakan fungsi-fungsi konstitusionalnya, seperti legislasi, anggaran, dan p

### [Deploy Streamlit ](https://ppw-berita.streamlit.app/)